In [4]:
%pip install -q cassio datasets langchain openai tiktoken

Note: you may need to restart the kernel to use updated packages.


In [5]:
pip install -U langchain-community

Note: you may need to restart the kernel to use updated packages.


In [6]:
from langchain.vectorstores.cassandra import Cassandra
from langchain.indexes.vectorstore import VectorStoreIndexWrapper
from langchain.llms import OpenAI
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.llms import Ollama
from langchain_community.embeddings import HuggingFaceEmbeddings


In [7]:
from datasets import load_dataset
import cassio

d:\langchain\venv\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [8]:
%pip install pyPDF2

In [9]:
from PyPDF2 import PdfReader

In [10]:
import os
from dotenv import load_dotenv

load_dotenv()  # Load variables from .env file

ASTRA_DB_APPLICATION_TOKEN = os.getenv("ASTRA_DB_APPLICATION_TOKEN")
ASTRA_DB_ID = os.getenv("ASTRA_DB_ID")

In [11]:
pdfreader = PdfReader("thecomedybible.pdf")

In [12]:
from typing_extensions import Concatenate
raw_text = ''
for i, page in enumerate(pdfreader.pages):
  content = page.extract_text()
  if content:
    raw_text += content

In [13]:
#raw_text

In [14]:
cassio.init(token=ASTRA_DB_APPLICATION_TOKEN, database_id=ASTRA_DB_ID)

In [15]:
llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1)
embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


C:\Users\mohit\AppData\Local\Temp\ipykernel_18040\2256349389.py:1: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="deepseek-r1:1.5b", temperature=0.1)
C:\Users\mohit\AppData\Local\Temp\ipykernel_18040\2256349389.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L

In [16]:
astra_vector_store = Cassandra(
    embedding=embedding,
    session=None,
    keyspace=None,
    table_name="qa_mini_demo",
)


In [17]:
from langchain.text_splitter import CharacterTextSplitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=800,
    chunk_overlap=200,
    length_function=len,
)

texts = text_splitter.split_text(raw_text)

In [18]:
astra_vector_store.add_texts(texts)
print("Inserted %i headlines." % len(texts))
astra_vector_index = VectorStoreIndexWrapper(vectorstore=astra_vector_store)

Inserted 882 headlines.


In [19]:
first_question = True
while True:
  if first_question:
      query_text = input("\nEnter your question(or type'quit' to exit):").strip()
  else:
    query_text = input("\nWhat's your next question(or type'quit' to exit):").strip()
  if query_text.lower() == "quit":
    break
  if query_text == "quit":
    continue
  first_question = False
  
  print("\nQUESTION:\"%s\"" % query_text)
  answer = astra_vector_index.query(query_text, llm=llm).strip()
  print("\nANSWER:\"%s\"\n" %answer)

  print("FIRST DOCUMENTS BY RELEVANCE:")
  for doc, score in astra_vector_store.similarity_search_with_score(query_text,k=4):
    print("[%0.4f] \"%s...\""%(score,doc.page_content))




QUESTION:"who is jerry seinfied"

ANSWER:"<think>
Okay, so I need to figure out who Jerry Seinfeld is based on the context provided. Let me read through the text again carefully.

The text starts with a statement about lottery tickets and how they have better odds of striking it rich compared to other lotteries. Then there are several paragraphs that talk about people getting jobs they didn't expect, including Conan O'Brien, who was replaced by Fritz Coleman on NBC's Late Night. There's also mention of Seinfeld being plucked from the stand-up circuit and working for Benson's sitcom at a salary of §4,000 a week.

Wait, I think I might have missed something earlier. Let me check again. The text does say that Seinfeld was plucked from the stand-up circuit in 1980 to work for Benson's sitcom. So that must be where he worked. 

I remember hearing about Jerry Seinfeld being a comedian and host who did work on some popular sitcoms, but I'm not exactly sure of the details. From what I recall,